## Coverting NER datasets to GPT-3 Datasets

### Inputs:
- BIO NER Dataset.
- Columns: ['tokens','ner_tags']
- Structure: "Word\tNER-Tag"
- Examples Separated by empty line

### Outputs:
- GPT-3 Ready Dataset. 
- Structure: "Prompt\tTestExample"

#### Parameters:
- Training data file
- Test data file
- Random seed
- \# of training samples per prompt
- Selection Strategy
- Prompt Structure

*For now only works with one entity type at a time  

In [1]:
import pandas as pd
import numpy as np
import ipdb
import pickle

In [2]:
pd.set_option('max_rows',500,'max_colwidth',500)

In [3]:
re_df = pd.read_csv('../data/ddi2013-type/train.no_mask.tsv',sep='\t')

In [4]:
re_df['doc'] = ['.'.join(s.split('.')[:2]) for s in re_df['index']]

In [5]:
dev_ids = pd.read_csv('../data/ddi2013-type/train.tsv',sep='\t')

In [6]:
re_df['split'] = 'dev'

for i in dev_ids['index'].values:
    re_df.loc[re_df['index'] == i,'split'] = 'train'

In [7]:
re_df['entity1'] = [s.split('|||')[1] for s in re_df['sentence']]
re_df['entity2'] = [s.split('|||')[2] for s in re_df['sentence']]
re_df['sentence'] = [s.split('|||')[0] for s in re_df['sentence']]

In [8]:
re_df.label.unique()

array(['DDI-effect', 'DDI-false', 'DDI-advise', 'DDI-mechanism',
       'DDI-int'], dtype=object)

In [266]:
data_name = 'ddi'
entity_type = 'Drug'
relation_type = 'Interaction'
seed = 21
few_shot_num = 5
number_of_test_samples = 50
selection_strategy = 'random'
permutation = 4

In [267]:
label_order = ['effect', 'false','advise','mechanism','int']

In [268]:
verbalizer = 'natural'# or ''

In [269]:
#Basic Prompt Structure

if verbalizer == '':
    prompt_sample_structure = 'Sentence: {}\n'+entity_type+'1: {}\n'+entity_type+'2: {}\n'+relation_type+': {}'
    empty_prompt_sample_structure = 'Sentence: {}\n'+entity_type+'1: {}\n'+entity_type+'2: {}\n'+relation_type+':'

    label_verbalizer = {'effect':'effect',
                        'false':'none',
                        'advise':'advice',
                        'mechanism':'mechanism',
                        'int':'other'
                       }
elif verbalizer == 'natural':
    #More Natural Prompt Structure
    # How are entities {} and {} related based on the previous sentence? 
    prompt_sample_structure = 'Sentence: {}\nHow do drugs {} and {} interact based on the previous sentence?\nInteraction: {}'
    empty_prompt_sample_structure = 'Sentence: {}\nHow do  drugs {} and {} interact based on the previous sentence?\nInteraction:'

    label_verbalizer = {'effect':"Effect Alteration",
                        'false':'No Interaction',
                        'advise':'Recommendation',
                        'mechanism':'Metabolic Mechanism ',
                        'int':'Unspecified'
                       }
else:
    #More Natural Prompt Structure
    # How are entities {} and {} related based on the previous sentence? 
    prompt_sample_structure = 'Sentence: {}\nHow do drugs {} and {} interact based on the previous sentence?\n{}'
    empty_prompt_sample_structure = 'Sentence: {}\nHow do  drugs {} and {} interact based on the previous sentence?\n'

    label_verbalizer = {'effect':'They have an effect on each other.',
                        'false':'They have no interaction.',
                        'advise':'Some medical advice is given about their interaction.',
                        'mechanism':'They interact through their metabolic mechanisms.',
                        'int':'They have some unspecified interaction.'
                       }

In [270]:
def add_prompts(df):
    """
    Combining sentences and entities to create prompts in Dataframe with 'sents' and 'entities' columns.
    Adds 'prompts' and 'empty_prompts' (prompt without answer) columns to DataFrame
    """
    
    prompts = []
    empty_prompts = []
    labels = []
    
    for i, row in df.iterrows():
        sent = row['sentence']
        entity1 = row['entity1']
        entity2 = row['entity2']
        label = row['label'].split('-')[1]
        label = label_verbalizer[label]
        
        labels.append(label)
        prompt = prompt_sample_structure.format(sent, entity1, entity2, label)
        empty_prompt = empty_prompt_sample_structure.format(sent, entity1, entity2)
        
        prompts.append(prompt)
        empty_prompts.append(empty_prompt)

    df['prompts'] = prompts
    df['empty_prompts'] = empty_prompts
    df['labels'] = labels
    unique_labels = set(labels)
    df['unique_labels'] = [unique_labels for _ in empty_prompts]
    
    return df

In [271]:
train = re_df[re_df['split'] == 'train']
dev = re_df[re_df['split'] == 'dev']

In [288]:
train.to_csv('ddi.train.tsv')
dev.to_csv('ddi.dev.tsv')

In [272]:
train = add_prompts(train)
dev = add_prompts(dev)

<ipython-input-270-af81c41910ca>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prompts'] = prompts
<ipython-input-270-af81c41910ca>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['empty_prompts'] = empty_prompts
<ipython-input-270-af81c41910ca>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [273]:
print(dev.prompts.values[1])

Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.
How do drugs aminoglycoside and antibiotics interact based on the previous sentence?
Interaction: Effect Alteration


In [274]:
def undersample(df, col_to_undersample, undersample_perc):
    
    new_df = []
    for label in label_order:
        
        g = df[df[col_to_undersample] == label_verbalizer[label]]
        print(label)
        if label == 'false':
            g = g[:int(len(g)*undersample_perc)]
        
        new_df.append(g)
        
    return pd.concat(new_df)

In [275]:
dev.groupby('labels').count()/len(dev)

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,unique_labels
labels,,,,,,,,,,
Effect Alteration,0.054666,0.054666,0.054666,0.054666,0.054666,0.054666,0.054666,0.054666,0.054666,0.054666
Metabolic Mechanism,0.051491,0.051491,0.051491,0.051491,0.051491,0.051491,0.051491,0.051491,0.051491,0.051491
No Interaction,0.861403,0.861403,0.861403,0.861403,0.861403,0.861403,0.861403,0.861403,0.861403,0.861403
Recommendation,0.026643,0.026643,0.026643,0.026643,0.026643,0.026643,0.026643,0.026643,0.026643,0.026643
Unspecified,0.005798,0.005798,0.005798,0.005798,0.005798,0.005798,0.005798,0.005798,0.005798,0.005798


In [276]:
dev = undersample(dev, 'labels',0.1)

effect
false
advise
mechanism
int


In [277]:
dev.groupby('labels').count()/len(dev)

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,unique_labels
labels,,,,,,,,,,
Effect Alteration,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243
Metabolic Mechanism,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115
No Interaction,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292
Recommendation,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550
Unspecified,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799


In [278]:
dev

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,labels,unique_labels
12,DDI-DrugBank.d57.s0.p0,In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.,DDI-effect,DDI-DrugBank.d57,dev,aminoglycoside,antibiotics,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and antibiotics interact based on the previous sentence?\nInteraction: Effect Alteration,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and antibiotics interact based on the previous sentence?\nInteraction:,Effect Alteration,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}"
13,DDI-DrugBank.d57.s0.p1,In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.,DDI-effect,DDI-DrugBank.d57,dev,aminoglycoside,penicillins,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and penicillins interact based on the previous sentence?\nInteraction: Effect Alteration,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and penicillins interact based on the previous sentence?\nInteraction:,Effect Alteration,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}"
14,DDI-DrugBank.d57.s0.p2,In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.,DDI-effect,DDI-DrugBank.d57,dev,aminoglycoside,cephalosporins,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and cephalosporins interact based on the previous sentence?\nInteraction: Effect Alteration,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and cephalosporins interact based on the previous sentence?\nInteraction:,Effect Alteration,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}"
198,DDI-DrugBank.d5.s19.p4,MAO Inhibitors: Studies in animals demonstrate that the acute toxicity of bupropion is enhanced by the MAO inhibitor phenelzine .,DDI-effect,DDI-DrugBank.d5,dev,bupropion,phenelzine,Sentence: MAO Inhibitors: Studies in animals demonstrate that the acute toxicity of bupropion is enhanced by the MAO inhibitor phenelzine .\nHow do drugs bupropion and phenelzine interact based on the previous sentence?\nInteraction: Effect Alteration,Sentence: MAO Inhibitors: Studies in animals demonstrate that the acute toxicity of bupropion is enhanced by the MAO inhibitor phenelzine .\nHow do drugs bupropion and phenelzine interact based on the previous sentence?\nInteraction:,Effect Alteration,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}"
249,DDI-DrugBank.d568.s5.p0,Epidemiological studies of the case-control and cohort design that have demonstrated an association between use of psychotropic drugs that interfere with serotonin reuptake and the occurrence of upper gastrointestinal bleeding have also shown that concurrent use of an NSAID or aspirin potentiated the risk of bleeding.,DDI-effect,DDI-DrugBank.d568,dev,psychotropic drugs,NSAID,Sentence: Epidemiological studies of the ca

In [279]:
def create_stratified_prompt_dataset(train_df, test_df, seed, few_shot_num, number_of_test_samples, selection_strategy):
    
    random = np.random.RandomState(seed)
    
    few_shot_prompt, chosen_prompt_ids = create_few_shot_prompt(train_df, random, few_shot_num, selection_strategy)
    
    test_df['test_ready_prompt'] = [few_shot_prompt+'\n\n'+empty_prompt for empty_prompt in test_df['empty_prompts']]
    
    if number_of_test_samples != 'all':
        test_percentage = number_of_test_samples / len(test_df)
    
        #Making sure the samples are the same as the first batch
        random = np.random.RandomState(42)
        
        chosen_test_ids = []
        for i in ['advise','effect','mechanism','false','int']:
            g = test_df[test_df['labels'] == label_verbalizer[i]]
            label_chosen_ids = random.permutation(g.index)[:int(test_percentage*len(g))]
            
            chosen_test_ids.extend(label_chosen_ids)
        
        chosen_test_df = test_df.loc[chosen_test_ids]
    else:
        chosen_test_df = test_df
        
    return {'seed':seed,'few_shot_prompt': few_shot_prompt, 'chosen_prompt_ids':chosen_prompt_ids,'test_df': chosen_test_df}

def create_few_shot_prompt(train_df, random, few_shot_num, selection_strategy):
    
    if selection_strategy == 'random':
        chosen_inds = []
        labels = train_df.label.unique()

        i=0
        while True:
            chosen_inds.append(random.permutation(train_df[train_df.label == labels[i%len(labels)]].index)[0])
            i+=1
            if len(chosen_inds) == few_shot_num:
                break

    #Permute over prompt order
    chosen_inds = np.array(chosen_inds)[(np.arange(len(chosen_inds)) + permutation)%len(chosen_inds)]
    
    few_shot_prompt_list = train_df.loc[chosen_inds, 'prompts'].values
    few_shot_prompt = '\n\n'.join(few_shot_prompt_list)
    
    return few_shot_prompt, chosen_inds

In [280]:
test_ready_prompt_dataset = create_stratified_prompt_dataset(train, dev, seed, few_shot_num, number_of_test_samples, selection_strategy)

In [281]:
print(test_ready_prompt_dataset['few_shot_prompt'])

Sentence: Etonogestrel may interact with the following medications: acetaminophen (Tylenol), antibiotics such as ampicillin and tetracycline, anticonvulsants (Dilantin, Phenobarbital, Tegretol, Trileptal, Topamax, Felbatol), antifungals (Gris-PEG, Nizoral, Sporanox), atorvastatin (Lipitor), clofibrate (Atromid-S), cyclosporine (Neoral, Sandimmune), HIV drugs classified as protease inhibitors (Agenerase, Crixivan, Fortovase, Invirase, Kaletra, Norvir, Viracept), morphine (Astramorph, Kadian, MS Contin), phenylbutazone, prednisolone (Prelone), rifadin (rifampin), St. Johns wort, temazepam, theophylline (Theo-Dur), and vitamin C.
How do drugs Etonogestrel and Kaletra interact based on the previous sentence?
Interaction: Unspecified

Sentence: The administration of local anesthetic solutions containing epinephrine or norepinephrine to patients receiving monoamine oxidase inhibitors or tricyclic antidepressants may produce severe, prolonged hypertension.
How do drugs norepinephrine and tric

In [282]:
test_ready_prompt_dataset['test_df']

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,labels,unique_labels,test_ready_prompt
4386,DDI-DrugBank.d48.s12.p2,The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.,DDI-advise,DDI-DrugBank.d48,dev,proton pump inhibitors,SPRYCEL,Sentence: The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.\nHow do drugs proton pump inhibitors and SPRYCEL interact based on the previous sentence?\nInteraction: Recommendation,Sentence: The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.\nHow do drugs proton pump inhibitors and SPRYCEL interact based on the previous sentence?\nInteraction:,Recommendation,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}","Sentence: Etonogestrel may interact with the following medications: acetaminophen (Tylenol), antibiotics such as ampicillin and tetracycline, anticonvulsants (Dilantin, Phenobarbital, Tegretol, Trileptal, Topamax, Felbatol), antifungals (Gris-PEG, Nizoral, Sporanox), atorvastatin (Lipitor), clofibrate (Atromid-S), cyclosporine (Neoral, Sandimmune), HIV drugs classified as protease inhibitors (Agenerase, Crixivan, Fortovase, Invirase, Kaletra, Norvir, Viracept), morphine (Astramorph, Kadian, ..."
17780,DDI-DrugBank.d31.s0.p0,MAO inhibitors should be used with caution in patients receiving hydralazine.,DDI-advise,DDI-DrugBank.d31,dev,MAO inhibitors,hydralazine,Sentence: MAO inhibitors should be used with caution in patients receiving hydralazine.\nHow do drugs MAO inhibitors and hydralazine interact based on the previous sentence?\nInteraction: Recommendation,Sentence: MAO inhibitors should be used with caution in patients receiving hydralazine.\nHow do drugs MAO inhibitors and hydralazine interact based on the previous sentence?\nInteraction:,Recommendation,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}","Sentence: Etonogestrel may interact with the following medications: acetaminophen (Tylenol), antibiotics such as ampicillin and tetracycline, anticonvulsants (Dilantin, Phenobarbital, Tegretol, Trileptal, Topamax, Felbatol), antifungals (Gris-PEG, Nizoral, Sporanox), atorvastatin (Lipitor), clofibrate (Atromid-S), cyclosporine (Neoral, Sandimmune), HIV drugs classified as protease inhibitors (Agenerase, Crixivan, Fortovase, Invirase, Kaletra, Norvir, Viracept), morphine (Astramorph, Kadian, ..."
9530,DDI-DrugBank.d165.s16.p2,"Cyclosporine, tacrolimus and digoxin concentrations should be monitored at the initiation of Itraconazole therapy and frequently thereafter, and the dose of these three drug products adjusted appropriately.",DDI-advise,DDI-DrugBank.d165,dev,Cyclosporine,Itraconazole,"Sentence: Cyclosporine, tacrolimus and digoxin concentrations should be monitored at the initiation of Itraconazole therapy and frequently thereafter, and the dose of these three drug products adjusted appropriately.\nHow do drugs Cyclosporine and Itraconazole interact based on the previous sentence?\nInteraction: Recommendation","Sentence: Cyclosporine, tacrolimus and digoxin concentrations should be monitored at the initiation of Itraconazole therapy and frequently thereafter, and the dose of these three drug products adjusted appropriately.\nHow do drugs Cyclosporine and Itraconazole interact based on the previous sentence?\nInteraction:",Recommendation,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}","Sentence: Etonogestrel may interact with the following medications: acetaminophen (Tylenol), antibiotics such as ampicillin and tetracycline, anticonvulsants (Dilantin, Phenobarbital, Tegretol, Trileptal, Topamax, Felbatol), antifungals (Gris-PEG, Nizoral, Sporanox), atorvastatin (Lipitor), clofibrate (Atromid-S), cyclosporine (Neoral, Sandimmune), HIV drugs classified as protease inhibitors (Agenerase, Crixivan, Fortovase, Invirase, Kaletra, Norv

In [283]:
test_ready_prompt_dataset['test_df'].groupby('labels').count()

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,unique_labels,test_ready_prompt
labels,,,,,,,,,,,
Effect Alteration,12,12,12,12,12,12,12,12,12,12,12
Metabolic Mechanism,11,11,11,11,11,11,11,11,11,11,11
No Interaction,19,19,19,19,19,19,19,19,19,19,19
Recommendation,5,5,5,5,5,5,5,5,5,5,5
Unspecified,1,1,1,1,1,1,1,1,1,1,1


In [284]:
prompt_filename = '../data/{}_test_prompts_{}_{}.{}.{}.perm_{}.{}'.format(data_name, selection_strategy,seed,few_shot_num,number_of_test_samples,permutation,verbalizer)
pickle.dump(test_ready_prompt_dataset,open(prompt_filename,'wb'))

In [285]:
full_dev_prompt_filename = '../data/{}_test_prompts_full_dev_{}_{}.{}.perm_{}'.format(data_name, selection_strategy,seed,few_shot_num, permutation)
pickle.dump(dev,open(full_dev_prompt_filename,'wb'))

In [286]:
dev

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,labels,unique_labels,test_ready_prompt
12,DDI-DrugBank.d57.s0.p0,In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.,DDI-effect,DDI-DrugBank.d57,dev,aminoglycoside,antibiotics,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and antibiotics interact based on the previous sentence?\nInteraction: Effect Alteration,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and antibiotics interact based on the previous sentence?\nInteraction:,Effect Alteration,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}","Sentence: Etonogestrel may interact with the following medications: acetaminophen (Tylenol), antibiotics such as ampicillin and tetracycline, anticonvulsants (Dilantin, Phenobarbital, Tegretol, Trileptal, Topamax, Felbatol), antifungals (Gris-PEG, Nizoral, Sporanox), atorvastatin (Lipitor), clofibrate (Atromid-S), cyclosporine (Neoral, Sandimmune), HIV drugs classified as protease inhibitors (Agenerase, Crixivan, Fortovase, Invirase, Kaletra, Norvir, Viracept), morphine (Astramorph, Kadian, ..."
13,DDI-DrugBank.d57.s0.p1,In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.,DDI-effect,DDI-DrugBank.d57,dev,aminoglycoside,penicillins,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and penicillins interact based on the previous sentence?\nInteraction: Effect Alteration,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and penicillins interact based on the previous sentence?\nInteraction:,Effect Alteration,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}","Sentence: Etonogestrel may interact with the following medications: acetaminophen (Tylenol), antibiotics such as ampicillin and tetracycline, anticonvulsants (Dilantin, Phenobarbital, Tegretol, Trileptal, Topamax, Felbatol), antifungals (Gris-PEG, Nizoral, Sporanox), atorvastatin (Lipitor), clofibrate (Atromid-S), cyclosporine (Neoral, Sandimmune), HIV drugs classified as protease inhibitors (Agenerase, Crixivan, Fortovase, Invirase, Kaletra, Norvir, Viracept), morphine (Astramorph, Kadian, ..."
14,DDI-DrugBank.d57.s0.p2,In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.,DDI-effect,DDI-DrugBank.d57,dev,aminoglycoside,cephalosporins,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and cephalosporins interact based on the previous sentence?\nInteraction: Effect Alteration,Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.\nHow do drugs aminoglycoside and cephalosporins interact based on the previous sentence?\nInteraction:,Effect Alteration,"{No Interaction, Unspecified, Metabolic Mechanism , Effect Alteration, Recommendation}","Sentence: Etonogestrel may interact with the following medications: acetaminophen (Tylenol), antibiotics such as ampicillin and tetracycline, anticonvulsants (Dilantin, Phenobarbital, Tegretol, Trileptal, Topamax, Felbatol), ant

In [242]:
prompt_filename

'../data/ddi_test_prompts_random_33.5.50.perm_4.natural'